In [ ]:
!pip install langchain sentence-transformers transformers faiss-gpu PyPDF2 torch llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 104.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

# New Section

In [ ]:
from langchain.document_loaders.text import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from huggingface_hub import InferenceClient

from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA,ConversationChain

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from huggingface_hub import hf_hub_download
from langchain.llms import LlamaCpp

import time

#from PyPDF2 import PdfReader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
loader = TextLoader("/content/drive/MyDrive/Colab Notebooks/hackathon_MoroccoAI/blog_data_1.txt")
pages = loader.load()

In [ ]:
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=150,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [ ]:
chunks_text = split_text(pages)

Split 1 documents into 637 chunks.
---


Establishing a thriving business in Casablanca, Morocco, requires navigating various legal and administrative procedures. One of the crucial steps in this process is registering your company with the Casablanca Company Registry (Registre du commerce de Casablanca). 
This guide provides you with a comprehensive overview of the company registration process in Casablanca, empowering you to navigate the steps with confidence.
{'source': '/content/drive/MyDrive/Colab Notebooks/hackathon_MoroccoAI/blog_data_1.txt', 'start_index': 6808}


In [ ]:
# def Pdf_to_text(path) :
#     pdf_reader = PdfReader(path)

#     text = ""
#     for page in pdf_reader.pages:
#         text += page.extract_text()

#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=200,
#         length_function=len
#         )
#     chunks = text_splitter.split_text(text=text)
#     return chunks

In [ ]:
#chunks_pdf = Pdf_to_text("./drive/MyDrive/Colab Notebooks/hackathon_MoroccoAI/Doing-Business-Guide-Morocco.pdf")

In [ ]:
#embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') # machi top

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
docs_text = [doc.page_content for doc in chunks_text]

In [ ]:
# final_chunks = []

# # for chunk in chunks_pdf :
# #   final_chunks.append(chunk)

# for chunk in docs_text :
#     final_chunks.append(chunk)

In [ ]:
VectorStore = FAISS.from_texts(docs_text, embedding=embedding)

In [ ]:
# B_INST, E_INST = "[INST]", "[/INST]"
# B_SYS, E_SYS = "<>\n", "\n<>\n\n"
# DEFAULT_SYSTEM_PROMPT = " "

# def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
#     SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
#     prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
#     return prompt_template

# instruction = "Given the context that has been provided. \n {context}, Answer the following question - \n{question}"
# system_prompt = """Hello dear user . I am an AI advisor for founders and future founders or everyone interested in doing business in Morocco.How can I help you today :D"""

# template = get_prompt(instruction, system_prompt)
# print(template)



In [ ]:
# prompt = PromptTemplate(template=template, input_variables=["context", "question"])
# prompt

In [ ]:
MODEL_ID = "TheBloke/Mistral-7B-OpenOrca-GGUF"
MODEL_BASENAME = "mistral-7b-openorca.Q4_K_M.gguf"

In [ ]:
model_path = hf_hub_download(
            repo_id=MODEL_ID,
            filename=MODEL_BASENAME,
            resume_download=True,
        )

mistral-7b-openorca.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


CONTEXT_WINDOW_SIZE = 1500
MAX_NEW_TOKENS = 2000
N_BATCH = 512
n_gpu_layers = 40
kwargs = {
          "model_path": model_path,
          "n_ctx": CONTEXT_WINDOW_SIZE,
          "max_tokens": MAX_NEW_TOKENS,
          "n_batch": N_BATCH,
          "n_gpu_layers": n_gpu_layers,
          "callback_manager": callback_manager,
          "verbose":True,
      }

In [ ]:
# from langchain.callbacks.manager import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain.chains import LLMChain
# from langchain.llms import LlamaCpp

# # Callbacks support token-wise streaming
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
# n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
# max_tokens = 2000
# # Make sure the model path is correct for your system!
# llm = LlamaCpp(
#     model_path=model_path,
#     n_gpu_layers=n_gpu_layers,

#     n_batch=n_batch,
#     max_tokens= max_tokens,
#     callback_manager=callback_manager,
#     verbose=True,  # Verbose is required to pass to the callback manager
# )


In [ ]:
llm = LlamaCpp(**kwargs)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key='question',
    output_key='answer'
)

In [ ]:
qa = ConversationalRetrievalChain.from_llm(
    llm,
    chain_type="stuff",
    retriever=VectorStore.as_retriever(search_kwargs={"k": 5}),
    memory=memory,
    return_source_documents=True,
    verbose=False,
)

In [ ]:
start = time.time()
res = qa(f"""
I'm intressted in starting the buisness in Casa , what I should do next?""")
end = time.time()
execution_time = end - start

KeyboardInterrupt: ignored

In [ ]:
start = time.time()
res = qa(f"""
Can you define all the buisness terminolgy you mentioned """)
end = time.time()
execution_time = end - start

In [ ]:
start = time.time()
res = qa(f"""
I want to start my new buisness in finance what, are some cities do you recommend for starting my buisness?
""")
end = time.time()
execution_time = end - start